In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'tranee_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 4317 images belonging to 5 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 4797 images belonging to 5 classes.


In [4]:
cnn = tf.keras.models.Sequential()

In [5]:
cnn.add(tf.keras.layers.Conv2D(filters=64 , kernel_size=3 , activation='relu' , input_shape=[64,64,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=64 , kernel_size=3 , activation='relu' ))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 , strides=2))

In [7]:
cnn.add(tf.keras.layers.Dropout(0.5))

In [8]:
cnn.add(tf.keras.layers.Flatten())

In [9]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [10]:
cnn.add(tf.keras.layers.Dense(units=5 , activation='softmax'))

In [11]:
cnn.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [12]:
cnn.fit(x = training_set , validation_data = test_set , epochs = 20)

Epoch 1/20
135/135 [==============================] - 48s 350ms/step - loss: 1.2952 - accuracy: 0.4496 - val_loss: 1.0927 - val_accuracy: 0.5777
Epoch 2/20
135/135 [==============================] - 48s 353ms/step - loss: 1.0850 - accuracy: 0.5724 - val_loss: 0.9360 - val_accuracy: 0.6358
Epoch 3/20
135/135 [==============================] - 49s 364ms/step - loss: 0.9708 - accuracy: 0.6245 - val_loss: 0.8667 - val_accuracy: 0.6729
Epoch 4/20
135/135 [==============================] - 44s 330ms/step - loss: 0.8993 - accuracy: 0.6507 - val_loss: 0.7962 - val_accuracy: 0.6990
Epoch 5/20
135/135 [==============================] - 52s 384ms/step - loss: 0.8536 - accuracy: 0.6699 - val_loss: 0.8561 - val_accuracy: 0.6656
Epoch 6/20
135/135 [==============================] - 52s 385ms/step - loss: 0.8057 - accuracy: 0.6850 - val_loss: 0.7205 - val_accuracy: 0.7263
Epoch 7/20
135/135 [==============================] - 49s 360ms/step - loss: 0.7666 - accuracy: 0.7084 - val_loss: 0.7581 - val_ac

In [20]:
from keras.preprocessing import image
test_image = image.load_img('Prediction/s.jpg',target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = cnn.predict(test_image)

1/1 [==============================] - 0s 23ms/step


In [21]:
training_set.class_indices

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}

In [22]:
print(result)

[[0. 0. 0. 1. 0.]]


In [23]:
if result[0][0]==1:
    print('Daisy')
elif result[0][1]==1:
    print('Dandelion')
elif result[0][2]==1:
    print('Rose')
elif result[0][3]==1:
    print('SunFlower')
elif result[0][4]==1:
    print("Tulip")

SunFlower


In [18]:
print(result)

[[1. 0. 0. 0. 0.]]
